In [1]:
#Import libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import rapidfuzz
import difflib
from nltk.corpus import stopwords
from yake import KeywordExtractor

In [2]:
Stops = stopwords.words('english') #Returns all stopwords in the English language. 
print(Stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Conneticut

In [3]:
State = 'CT'
AllOccs = pd.read_csv('../../../Data/Post1790/aggregated_CD_noname.csv') #Returns aggregated CSV file. 
AllOccs.drop(columns=['Unnamed: 0'], inplace=True) #Drop duplicate index column which appears.
AllOccsCT = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsCT = AllOccsCT.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsCT)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       1064.0      75.0          532.0          37.0      508.0      51.0   
1        449.0      96.0          224.0          97.0      232.0      10.0   
2        196.0      75.0           98.0          37.0      172.0      24.0   
3         53.0      58.0           26.0          79.0       67.0       6.0   
4        121.0      22.0           60.0          61.0       32.0      72.0   
..         ...       ...            ...           ...        ...       ...   
478       39.0      11.0           19.0          55.0       14.0       7.0   
479       13.0      21.0            6.0          61.0        4.0      74.0   
480      533.0      34.0          266.0          66.0      446.0      66.0   
481      658.0      47.0          329.0          23.0      558.0      50.0   
482       12.0      22.0            6.0          11.0       11.0       NaN   

           town state occupation             county name_type  

In [4]:
OccsTotalsCT = AllOccsCT.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsCT['# of people'] = OccsTotalsCT['occupation'].map(AllOccsCT['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsCT.to_string())

                                                           occupation  6p_total  # of people
0                                   Adminastrator of the Asa Waterman    218.47            1
1                                      Adminastrator to David Fellows      0.00            1
2   Adminastrator to the Estate of Epaphras Bull of Hartford Deceased     61.30            1
3                        Adminastrator to the Estate of Samuel Tilley     94.67            1
4            Adminastrator to the estate of Uriah Loomis Jun Deceased     73.50            1
5                                          Administrator to J McCurdy   3413.72            2
6                         Administrator to the estate of Daniel Unkus     54.05            1
7                Adminstrator to the Estate of Ezekiel Carey Deceased     77.61            1
8                  Adminstrator to the Estate of Ruth Skiffe Deceased     87.74            1
9                                                          Apothecary 

In [5]:
#Function that returns the same dataframe, but with all stopwords removed from the occupation titles. 
def RemoveStopwords(OccsTotals):
    OccsTotals['occupation'] = OccsTotals['occupation'].apply(lambda OccName: ' '.join(word.lower() for word in OccName.split() if word not in Stops))
    return OccsTotals 

In [6]:
OccsTotalsCT = RemoveStopwords(OccsTotalsCT) #Call 
print(OccsTotalsCT.to_string())

                                              occupation  6p_total  # of people
0                             adminastrator asa waterman    218.47            1
1                            adminastrator david fellows      0.00            1
2   adminastrator estate epaphras bull hartford deceased     61.30            1
3                     adminastrator estate samuel tilley     94.67            1
4         adminastrator estate uriah loomis jun deceased     73.50            1
5                                administrator j mccurdy   3413.72            2
6                      administrator estate daniel unkus     54.05            1
7             adminstrator estate ezekiel carey deceased     77.61            1
8               adminstrator estate ruth skiffe deceased     87.74            1
9                                             apothecary      7.72            1
10                                              attorney   4863.67            3
11                                      

In [7]:
Remove = 2 #Define remove constant. 

In [8]:
#This function will help determine how many words we can acceptably cut off from the end. 
def FindRemoveConst(Occupations):
    TempMaxLength = 2
    
    print('Occupation Before --> Occupation After')
    print('--------------------------------------')
    print('\n')
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > TempMaxLength): #If occupation has more than two words, remove last two words.
            OccupationAfter = Occupation.rsplit(' ', Remove)[0] 
        else:
            OccupationAfter = Occupation #Else, set them both equal to each other; nothing changes. 
        
        print(Occupation + ' --> ' + OccupationAfter)

In [9]:
FindRemoveConst(OccsTotalsCT['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


adminastrator asa waterman --> adminastrator
adminastrator david fellows --> adminastrator
adminastrator estate epaphras bull hartford deceased --> adminastrator estate epaphras bull
adminastrator estate samuel tilley --> adminastrator estate
adminastrator estate uriah loomis jun deceased --> adminastrator estate uriah loomis
administrator j mccurdy --> administrator
administrator estate daniel unkus --> administrator estate
adminstrator estate ezekiel carey deceased --> adminstrator estate ezekiel
adminstrator estate ruth skiffe deceased --> adminstrator estate ruth
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney law
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier hartford bank --> cashier
clothier --> clothier
committee 1st ecclestical society --> committee 1st
committee 1st society plainville --> committee 1st
cooper --> cooper
doctor --> doctor
esqui

In [10]:
#Declare necessary parameters for KeywordExtractor() function.
MaxLength = 2
language = 'en' #Set language to English.
MaxSize = 1  #Each keyword can only be a max. length of 1.
DupThres = 0.5 #Words can not be used repeatedly in keywords. Treat each word separately. 
KeyExtractor = KeywordExtractor(lan=language, n=MaxSize, dedupLim=DupThres, features=None) #Create KeywordExtractor object.
NoMerge = []

In [11]:
# Manually clean items. 
Exceptions = {'inn keeper': 'inn keeper', 'leather dresser': 'leather dresser', 'post rider': 'post rider', 
              'school committee': 'school committee', 'school master': 'school master', 'society committee': 'committee', 
              'society treasurer': 'treasurer', 'frederick company': 'frederick company', 'the state north carolina': None,
              'the state of north carolina' : 'the state of north carolina','in trust jacobus van eis somerset county': 'assignee', 
              'ah.vat law': 'attorney', 'black smith': 'blacksmith','book binder': 'book binder', 'ex estate john baird': 'executors', 
              'ex': 'executors', 'junior broker': 'broker', 'post master': 'post master', 'ship wright': 'ship wright', 'shop keeper': 'shopkeeper', 
              'christopher mary': 'individual', 'law': 'attorney', 'lawyer': 'attorney', 
              'adm alexander mccaraher late chester': 'administer', 
              'general treasurer state rhode island suceessors said': 'treasurer', 'behalf proprietors': 'proprietors', 
              'chaise maker': 'chaise maker', 'executrix william': 'executor', 'blacks smith': 'blacksmith', 
              'cabinet maker': 'cabinet maker', 'charleston merchants': 'merchants', 'hair dresser': 'hair dresser', 
              'south carolina': 'south carolina', 'vendue master': 'auctioneer', 'physician' : 'doctor', 'administer' : 'adminastrator',
                'esquier' : 'esq', 'excuters' : 'excuetor', 'extors' : 'excuetor', 'esquire' : 'esq', 'executers' : 'excuetor',
             'yeoman' : 'farmer', 'cordwainer' : 'shoemaker', 'joiner' : 'carpenter', 'innholder' : 'inn keeper', 'factor' : 'trader',
             'asse john b ashe exe' : 'executors', 'trust' : 'assignee', 'pennsylvania' : None, 'co' : None, 'penniston' : None}

In [12]:
#Print alphabetized version of 'Exceptions'.
for Name in sorted(Exceptions):
    print("%s: %s" % (Name, Exceptions[Name]))

adm alexander mccaraher late chester: administer
administer: adminastrator
ah.vat law: attorney
asse john b ashe exe: executors
behalf proprietors: proprietors
black smith: blacksmith
blacks smith: blacksmith
book binder: book binder
cabinet maker: cabinet maker
chaise maker: chaise maker
charleston merchants: merchants
christopher mary: individual
co: None
cordwainer: shoemaker
esquier: esq
esquire: esq
ex: executors
ex estate john baird: executors
excuters: excuetor
executers: excuetor
executrix william: executor
extors: excuetor
factor: trader
frederick company: frederick company
general treasurer state rhode island suceessors said: treasurer
hair dresser: hair dresser
in trust jacobus van eis somerset county: assignee
inn keeper: inn keeper
innholder: inn keeper
joiner: carpenter
junior broker: broker
law: attorney
lawyer: attorney
leather dresser: leather dresser
penniston: None
pennsylvania: None
physician: doctor
post master: post master
post rider: post rider
school committee: 

In [13]:
#Search for an item in 'Exceptions' dictionary. Uncomment when necessary.
"""
Item = input()
if (Item in Exceptions) or (Item in Exceptions.values()): #Check keys and values of the dictionary. 
    print(True)
else:
    print(False)
"""

'\nItem = input()\nif (Item in Exceptions) or (Item in Exceptions.values()): #Check keys and values of the dictionary. \n    print(True)\nelse:\n    print(False)\n'

In [14]:
#Print what the merged occupations will look like. This will help us find any exceptions. 
def PrintFutureOccupations(Occupations):    
    print('Occupations --> Potential Cleaned Occupations')
    print('---------------------------------------------')
    print('\n')
        
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', Remove)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        if (len(Keywords) == 0):
            Keywords.append((Occupation, ' '))
                    
        print(Occupation + ' --> ' + str(Keywords[0][0])) #Print current occupation title to hypothetical cleaned occupation title.

In [15]:
PrintFutureOccupations(OccsTotalsCT['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


adminastrator --> adminastrator
adminastrator --> adminastrator
adminastrator estate epaphras bull --> adminastrator
adminastrator estate --> adminastrator
adminastrator estate uriah loomis --> adminastrator
administrator --> administrator
administrator estate --> administrator
adminstrator estate ezekiel --> adminstrator
adminstrator estate ruth --> adminstrator
apothecary --> apothecary
attorney --> attorney
attorney law --> attorney
blacksmith --> blacksmith
broker --> broker
carpenter --> carpenter
cashier --> cashier
clothier --> clothier
committee 1st --> committee
committee 1st --> committee
cooper --> cooper
doctor --> doctor
esquire --> esquire
executor ezra selden --> executor
executor stephen --> executor
executor stephen --> executor
executor william --> executor
executor estate tim --> executor
executor estate uriah --> executor
executors estate jabez --> executors
farmer --> farm

In [16]:
#Handle duplicate occupations that are spelled differently. Pass in 'occupation' column as a list. Pass in function which will do a manual check for each state (differs w/ each state). 
def MergeOccupations(OccsTotals):
    OccsTotals['% of total people'] = None #Add a percent of total population column.
    Occupations = OccsTotals['occupation'].tolist() #Returns 'occupation' column as a list.    
    Index = 0 #Keep track of index. 
    
    for Occupation in Occupations:
        if (len(Occupation.split()) > MaxLength): #If occupation has more than two words in it, remove last two words. Depends on constant. Might change. 
            Occupation = Occupation.rsplit(' ', Remove)[0]
            
        Keywords = KeyExtractor.extract_keywords(Occupation) #Retrieve list of keywords. 
        
        if (len(Keywords) == 0):
            Keywords.append((Occupation, ' '))
        
        NewOccupation = Keywords[0][0] #Get new occupation from list of keywords. Based on which keyword has the highest rating.
                
        if (Occupation in Exceptions): #If occupation is an exception, set new occupation. 
            NewOccupation = Exceptions[Occupation]
    
        OccsTotals.loc[Index, 'occupation'] = NewOccupation #Replace occupation with new occupation in dataframe.
        
        Index += 1 #Increment. 
    
    print('After Cleaning')
    print('--------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
        
    OccsTotals = OccsTotals.groupby('occupation')[['6p_total', '# of people', '% of total people']].apply(sum).reset_index() #Combine duplicate occupations, reset index. 
    OccsTotals['# of people'] = OccsTotals['# of people'].astype(int) #Convert values in '# of people' column to all ints. 
    
    print('After Removing Duplicates and Summing')
    print('-------------------------------------')
    print('\n')
    print(OccsTotals.to_string()) #Check
    print('\n')
    
    Occupations = OccsTotals['occupation'].tolist() #Reinstantiate Occupations after changing OccsTotals
    i = 0 #Declare new index.
    while (i < len(Occupations)): 
        j = 0 #Declare new index.
        while (j < len(Occupations)):                
            if (rapidfuzz.fuzz.ratio(Occupations[i], Occupations[j]) > 80) and (Occupations[i] != Occupations[j]) and ((Occupations[j] not in NoMerge) and (Occupations[i] not in NoMerge))  : #Use fuzzy string matching to determine whether two occupations are the same or not.               
                Total = float(OccsTotals.loc[j, '6p_total']) #Get total from duplicate occupation.
                OccsTotals.loc[i, '6p_total'] += Total #Merge totals together.
                
                NumOfPeople = OccsTotals.loc[j, '# of people'] #Get number of people in row j.
                OccsTotals.loc[i, '# of people'] += NumOfPeople #Add that number to row i.
                
                OccsTotals.drop(j, inplace=True) #Remove duplicate occupation.
                OccsTotals = OccsTotals.reset_index(drop=True) #Reset index.
                
                print('Removed: ' + Occupations[j] + ' --> ' + Occupations[i])
                
                Occupations.remove(Occupations[j]) #Remove from list of occupations so loop will not iterate over it again.
            else:
                j += 1
                
        i += 1
        
    OccsTotals['average debt per person'] = OccsTotals['6p_total'] / OccsTotals['# of people'] #Add an average column.
    OccsTotals['% of total people'] = (OccsTotals['# of people'] / OccsTotals['# of people'].sum()) * 100 #Assign values to '# of people column'.
    OccsTotals = OccsTotals.sort_values('6p_total', ascending=False).reset_index(drop=True)
    
    print('\n')
    print('After Using Fuzzy String Matching')
    print('---------------------------------')
    print('\n')
    print(OccsTotals.to_string())
    print('\n')
        
    return OccsTotals
        

In [17]:
OccsTotalsCT = MergeOccupations(OccsTotalsCT)

After Cleaning
--------------


          occupation  6p_total  # of people % of total people
0      adminastrator    218.47            1              None
1      adminastrator      0.00            1              None
2      adminastrator     61.30            1              None
3      adminastrator     94.67            1              None
4      adminastrator     73.50            1              None
5      administrator   3413.72            2              None
6      administrator     54.05            1              None
7       adminstrator     77.61            1              None
8       adminstrator     87.74            1              None
9         apothecary      7.72            1              None
10          attorney   4863.67            3              None
11          attorney   6250.93           17              None
12        blacksmith    724.71            3              None
13            broker    208.55            3              None
14         carpenter    175.34        

In [18]:
def TurnToExcel(DF, State):
    DF.to_csv('./FinishedSpreadsheets/Occupations' + State + '.csv')

In [19]:
TurnToExcel(OccsTotalsCT, State)

# Georgia

In [20]:
State = 'GA'
AllOccsGA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsGA = AllOccsGA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsGA)

   6p_Dollar    6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0     4586.0     66.0000         2293.0       33.0000     3026.0     11.00   
1        1.0     95.1133            1.0       47.5567        NaN      0.00   
2        1.0     90.3000            1.0       45.1000        1.0     93.50   
3        1.0    129.6533            1.0       64.8233        1.0    144.96   
4        1.0    209.8633            1.0      104.9333        1.0    241.38   
5        1.0  10108.9033            1.0     5054.4533        1.0   2729.40   
6        1.0    114.7330            1.0       57.3330        1.0    124.45   
7       20.0     26.1200           16.0       13.0600        NaN       NaN   
8       37.0     18.2000           18.0        9.0900        NaN      0.00   
9    10508.0     90.0000            NaN           NaN        NaN       NaN   

               town state occupation           county name_type      6p_total  
0  State of Georgia    GA  Gentleman              NaN     sta

In [21]:
OccsTotalsGA = AllOccsGA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsGA['# of people'] = OccsTotalsGA['occupation'].map(AllOccsGA['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsGA.to_string())

  occupation      6p_total  # of people
0        Esq    102.089033            1
1  Gentleman   4586.660000            1
2   Merchant  10538.261296            6
3  Merchants     39.478533            2


In [22]:
OccsTotalsGA = RemoveStopwords(OccsTotalsGA) 
print(OccsTotalsGA.to_string())

  occupation      6p_total  # of people
0        esq    102.089033            1
1  gentleman   4586.660000            1
2   merchant  10538.261296            6
3  merchants     39.478533            2


In [23]:
FindRemoveConst(OccsTotalsGA['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


esq --> esq
gentleman --> gentleman
merchant --> merchant
merchants --> merchants


In [24]:
OccsTotalsGA = MergeOccupations(OccsTotalsGA)

After Cleaning
--------------


  occupation      6p_total  # of people % of total people
0        esq    102.089033            1              None
1  gentleman   4586.660000            1              None
2   merchant  10538.261296            6              None
3  merchants     39.478533            2              None


After Removing Duplicates and Summing
-------------------------------------


  occupation      6p_total  # of people  % of total people
0        esq    102.089033            1                  0
1  gentleman   4586.660000            1                  0
2   merchant  10538.261296            6                  0
3  merchants     39.478533            2                  0


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


  occupation      6p_total  # of people  % of total people  average debt per person
0   merchant  10577.739829            8               80.0              1322.217479
1  gentleman   4586.660000   

In [25]:
TurnToExcel(OccsTotalsGA, State)

# Maryland

In [26]:
State = 'MD'
AllOccsMD = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsMD = AllOccsMD.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsMD)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        17.0   66.5733           13.0       33.2833       14.0      76.4   
1     26745.0   77.0000        13372.0       88.0000    50000.0      51.0   
2      1243.0   42.0000          621.0       71.0000     1796.0      44.0   
3       117.0   90.0000           58.0       94.0000      113.0      53.0   
4       141.0    3.0000           70.0       51.0000      375.0      32.0   
5        40.0   81.0000           20.0       40.0000       40.0      79.0   
6       789.0   24.0000          394.0       61.0000        NaN       NaN   
7       812.0   33.0000          406.0       16.0000      999.0      45.0   
8       341.0    8.0000          170.0       53.0000      265.0      47.0   
9      1122.0   48.0000          561.0       24.0000      761.0      65.0   
10     1926.0   70.0000          963.0       35.0000      696.0      98.0   
11     3120.0   91.0000         1560.0       45.0000     2499.0      70.0   

In [27]:
OccsTotalsMD = AllOccsMD.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsMD['# of people'] = OccsTotalsMD['occupation'].map(AllOccsMD['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsMD.to_string())

                         occupation      6p_total  # of people
0     Administrator of John Johnson     22.500000            1
1   Adminstrator of Robert Whitaker      0.000000            1
2                               Esq     17.665733            1
3            Executor of Moses Ruth     44.840000            1
4        Executor of Sabret Sollars    141.030000            1
5        Executor of Thomas Sollars    117.900000            1
6                 Frederick Company    206.670000            1
7                          Merchant  48636.870000           45
8                         Merchants  29874.070000            2
9                         Shoemaker   1237.070000            2
10                       Shopkeeper     33.330000            1


In [28]:
OccsTotalsMD = RemoveStopwords(OccsTotalsMD) 
print(OccsTotalsMD.to_string())

                      occupation      6p_total  # of people
0     administrator john johnson     22.500000            1
1   adminstrator robert whitaker      0.000000            1
2                            esq     17.665733            1
3            executor moses ruth     44.840000            1
4        executor sabret sollars    141.030000            1
5        executor thomas sollars    117.900000            1
6              frederick company    206.670000            1
7                       merchant  48636.870000           45
8                      merchants  29874.070000            2
9                      shoemaker   1237.070000            2
10                    shopkeeper     33.330000            1


In [29]:
FindRemoveConst(OccsTotalsMD['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


administrator john johnson --> administrator
adminstrator robert whitaker --> adminstrator
esq --> esq
executor moses ruth --> executor
executor sabret sollars --> executor
executor thomas sollars --> executor
frederick company --> frederick company
merchant --> merchant
merchants --> merchants
shoemaker --> shoemaker
shopkeeper --> shopkeeper


In [30]:
PrintFutureOccupations(OccsTotalsMD['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator --> administrator
adminstrator --> adminstrator
esq --> esq
executor --> executor
executor --> executor
executor --> executor
frederick company --> frederick
merchant --> merchant
merchants --> merchants
shoemaker --> shoemaker
shopkeeper --> shopkeeper


In [31]:
OccsTotalsMD = MergeOccupations(OccsTotalsMD)

After Cleaning
--------------


           occupation      6p_total  # of people % of total people
0       administrator     22.500000            1              None
1        adminstrator      0.000000            1              None
2                 esq     17.665733            1              None
3            executor     44.840000            1              None
4            executor    141.030000            1              None
5            executor    117.900000            1              None
6   frederick company    206.670000            1              None
7            merchant  48636.870000           45              None
8           merchants  29874.070000            2              None
9           shoemaker   1237.070000            2              None
10         shopkeeper     33.330000            1              None


After Removing Duplicates and Summing
-------------------------------------


          occupation      6p_total  # of people  % of total people
0      administra

In [32]:
TurnToExcel(OccsTotalsMD, State)

# North Carolina

In [33]:
State = 'NC'
AllOccsNC = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNC = AllOccsNC.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNC)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        85.0      25.0           42.0          62.0       41.0      82.0   
1      2008.0      37.0         1004.0          18.0     1324.0      76.0   
2        48.0      50.0           24.0          25.0        NaN       NaN   
3       176.0       NaN           88.0           NaN      150.0      67.0   
4     12283.0      44.0         6141.0          71.0    10781.0      94.0   
5        41.0      26.0           20.0          64.0       15.0      83.0   
6       206.0       8.0          103.0           5.0       50.0      69.0   
7       228.0      12.0          114.0           6.0      157.0      80.0   
8       252.0      32.0          126.0          16.0        NaN       NaN   
9      4780.0      84.0         2390.0          41.0     4998.0      96.0   
10        0.0       0.0          892.0          66.0     1339.0       NaN   
11       62.0      51.0           31.0          26.0        NaN       NaN   

In [34]:
OccsTotalsNC = AllOccsNC.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNC['# of people'] = OccsTotalsNC['occupation'].map(AllOccsNC['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNC.to_string())

                    occupation  6p_total  # of people
0      Asse of John B Ashe Exe      0.00            1
1            Doctors of Physic   2008.37            1
2                       Farmer     62.51            1
3                     Merchant    813.03            5
4                    Merchants     48.50            2
5  The State of North Carolina   4780.84            1
6                    Treasurer  12283.44            1


In [35]:
OccsTotalsNC = RemoveStopwords(OccsTotalsNC) 
print(OccsTotalsNC.to_string())

                 occupation  6p_total  # of people
0      asse john b ashe exe      0.00            1
1            doctors physic   2008.37            1
2                    farmer     62.51            1
3                  merchant    813.03            5
4                 merchants     48.50            2
5  the state north carolina   4780.84            1
6                 treasurer  12283.44            1


In [36]:
FindRemoveConst(OccsTotalsNC['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


asse john b ashe exe --> asse john b
doctors physic --> doctors physic
farmer --> farmer
merchant --> merchant
merchants --> merchants
the state north carolina --> the state
treasurer --> treasurer


In [37]:
Remove = 0

In [38]:
PrintFutureOccupations(OccsTotalsNC['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


asse john b ashe exe --> asse
doctors physic --> doctors
farmer --> farmer
merchant --> merchant
merchants --> merchants
the state north carolina --> carolina
treasurer --> treasurer


In [39]:
OccsTotalsNC = MergeOccupations(OccsTotalsNC)

After Cleaning
--------------


  occupation  6p_total  # of people % of total people
0  executors      0.00            1              None
1    doctors   2008.37            1              None
2     farmer     62.51            1              None
3   merchant    813.03            5              None
4  merchants     48.50            2              None
5       None   4780.84            1              None
6  treasurer  12283.44            1              None


After Removing Duplicates and Summing
-------------------------------------


  occupation  6p_total  # of people  % of total people
0    doctors   2008.37            1                  0
1  executors      0.00            1                  0
2     farmer     62.51            1                  0
3   merchant    813.03            5                  0
4  merchants     48.50            2                  0
5  treasurer  12283.44            1                  0


Removed: merchants --> merchant


After Using Fuzzy String Matching
-

In [40]:
TurnToExcel(OccsTotalsNC, State)

# New Hampshire

In [41]:
State = 'NH'
AllOccsNH = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNH = AllOccsNH.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNH)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        200.0       NaN          100.0           NaN       54.0       NaN   
1        778.0       4.0          389.0           2.0      288.0       NaN   
2       1490.0      73.0          745.0          37.0      436.0      77.0   
3       2100.0      48.0         1050.0          23.0      928.0      46.0   
4        149.0      62.0           74.0          81.0        NaN       NaN   
..         ...       ...            ...           ...        ...       ...   
161       81.0      69.0           40.0          84.0      741.0      54.0   
162      187.0       2.0           93.0          51.0      146.0      70.0   
163       12.0      73.0            6.0          37.0        0.0       0.0   
164        9.0      16.0            4.0          58.0       10.0      16.0   
165       40.0      67.0           20.0          33.0       25.0      96.0   

                town state occupation             county name_t

In [42]:
OccsTotalsNH = AllOccsNH.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNH['# of people'] = OccsTotalsNH['occupation'].map(AllOccsNH['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNH.to_string())

    occupation  6p_total  # of people
0        Baker   2159.04            1
1   Blacksmith     54.88            1
2        Clerk    715.60            7
3   Cordwainer    742.05            2
4      Esquire  61539.89           66
5       Farmer    778.04            1
6    Gentleman   3603.06            6
7    Goldsmith    871.32            1
8       Joiner    106.61            2
9     Marniner    785.62            2
10    Merchant  29715.41           26
11     Painter     48.61            1
12   Physician   7111.43           18
13    Spinster     89.13            1
14      Trader   2807.27            8
15   Treasurer    149.62            1
16       Widow   8790.45           10
17      Yeoman   3397.94           12


In [43]:
OccsTotalsNH = RemoveStopwords(OccsTotalsNH) 
print(OccsTotalsNH.to_string())

    occupation  6p_total  # of people
0        baker   2159.04            1
1   blacksmith     54.88            1
2        clerk    715.60            7
3   cordwainer    742.05            2
4      esquire  61539.89           66
5       farmer    778.04            1
6    gentleman   3603.06            6
7    goldsmith    871.32            1
8       joiner    106.61            2
9     marniner    785.62            2
10    merchant  29715.41           26
11     painter     48.61            1
12   physician   7111.43           18
13    spinster     89.13            1
14      trader   2807.27            8
15   treasurer    149.62            1
16       widow   8790.45           10
17      yeoman   3397.94           12


In [44]:
FindRemoveConst(OccsTotalsNH['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


baker --> baker
blacksmith --> blacksmith
clerk --> clerk
cordwainer --> cordwainer
esquire --> esquire
farmer --> farmer
gentleman --> gentleman
goldsmith --> goldsmith
joiner --> joiner
marniner --> marniner
merchant --> merchant
painter --> painter
physician --> physician
spinster --> spinster
trader --> trader
treasurer --> treasurer
widow --> widow
yeoman --> yeoman


In [45]:
PrintFutureOccupations(OccsTotalsNH['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


baker --> baker
blacksmith --> blacksmith
clerk --> clerk
cordwainer --> cordwainer
esquire --> esquire
farmer --> farmer
gentleman --> gentleman
goldsmith --> goldsmith
joiner --> joiner
marniner --> marniner
merchant --> merchant
painter --> painter
physician --> physician
spinster --> spinster
trader --> trader
treasurer --> treasurer
widow --> widow
yeoman --> yeoman


In [46]:
OccsTotalsNH = MergeOccupations(OccsTotalsNH)

After Cleaning
--------------


    occupation  6p_total  # of people % of total people
0        baker   2159.04            1              None
1   blacksmith     54.88            1              None
2        clerk    715.60            7              None
3    shoemaker    742.05            2              None
4          esq  61539.89           66              None
5       farmer    778.04            1              None
6    gentleman   3603.06            6              None
7    goldsmith    871.32            1              None
8    carpenter    106.61            2              None
9     marniner    785.62            2              None
10    merchant  29715.41           26              None
11     painter     48.61            1              None
12      doctor   7111.43           18              None
13    spinster     89.13            1              None
14      trader   2807.27            8              None
15   treasurer    149.62            1              None
16       widow  

In [47]:
TurnToExcel(OccsTotalsNH, State)

# New Jersey

In [48]:
State = 'NJ'
AllOccsNJ = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNJ = AllOccsNJ.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNJ)

   6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0     1324.0      81.0          662.0          41.0      675.0      52.0   
1     1325.0      68.0          662.0          85.0      714.0      50.0   
2      384.0      34.0          192.0          65.0      156.0      34.0   
3     5017.0       5.0         2508.0          52.0     2053.0      54.0   
4     1046.0      10.0          523.0           5.0      297.0      74.0   
5      235.0       1.0          117.0          51.0      139.0      10.0   
6        3.0      34.0            1.0          66.0        2.0      90.0   
7      184.0       NaN           92.0           NaN      437.0       6.0   
8       16.0      67.0            8.0          33.0        8.0      57.0   

                         town state  \
0                  New Jersey    NJ   
1                  New Jersey    NJ   
2                  New Jersey    NJ   
3                  New Jersey    NJ   
4                  New Jersey    NJ   
5  Mo

In [49]:
OccsTotalsNJ = AllOccsNJ.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNJ['# of people'] = OccsTotalsNJ['occupation'].map(AllOccsNJ['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNJ.to_string())

                                                   occupation  6p_total  # of people
0                                             Attorney at law   1325.68            1
1                                                     Esquire      0.00            1
2                        Executor of Michael De Mott deceased   1324.81            1
3                                                      Farmer      3.34            1
4                                                   Gentleman   6063.15            2
5  In trust for Jacobus Van Eis of Somerset County New Jersey     16.67            1
6                                                    Merchant    235.01            1
7                                                   Physician    384.34            1


In [50]:
OccsTotalsNJ = RemoveStopwords(OccsTotalsNJ) 
print(OccsTotalsNJ.to_string())

                                            occupation  6p_total  # of people
0                                         attorney law   1325.68            1
1                                              esquire      0.00            1
2                    executor michael de mott deceased   1324.81            1
3                                               farmer      3.34            1
4                                            gentleman   6063.15            2
5  in trust jacobus van eis somerset county new jersey     16.67            1
6                                             merchant    235.01            1
7                                            physician    384.34            1


In [51]:
FindRemoveConst(OccsTotalsNJ['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


attorney law --> attorney law
esquire --> esquire
executor michael de mott deceased --> executor michael de mott deceased
farmer --> farmer
gentleman --> gentleman
in trust jacobus van eis somerset county new jersey --> in trust jacobus van eis somerset county new jersey
merchant --> merchant
physician --> physician


In [52]:
Remove = 2

In [53]:
PrintFutureOccupations(OccsTotalsNJ['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


attorney law --> attorney
esquire --> esquire
executor michael de --> executor
farmer --> farmer
gentleman --> gentleman
in trust jacobus van eis somerset county --> county
merchant --> merchant
physician --> physician


In [54]:
OccsTotalsNJ = MergeOccupations(OccsTotalsNJ)

After Cleaning
--------------


  occupation  6p_total  # of people % of total people
0   attorney   1325.68            1              None
1        esq      0.00            1              None
2   executor   1324.81            1              None
3     farmer      3.34            1              None
4  gentleman   6063.15            2              None
5   assignee     16.67            1              None
6   merchant    235.01            1              None
7     doctor    384.34            1              None


After Removing Duplicates and Summing
-------------------------------------


  occupation  6p_total  # of people  % of total people
0   assignee     16.67            1                  0
1   attorney   1325.68            1                  0
2     doctor    384.34            1                  0
3        esq      0.00            1                  0
4   executor   1324.81            1                  0
5     farmer      3.34            1                  0
6  gentleman   60

In [55]:
TurnToExcel(OccsTotalsNJ, State)

# New York

In [56]:
State = 'NY'
AllOccsNY = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsNY = AllOccsNY.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsNY)

    6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       131.0   15.0000           65.0       57.0000      146.0     69.00   
1       118.0   45.0000           59.0       23.0000      124.0     94.00   
2        23.0   69.0000           11.0       84.0000       24.0     99.00   
3       261.0   31.0000          130.0       65.0000      340.0     53.00   
4       933.0   33.0000          466.0       67.0000      781.0     67.00   
5       555.0   86.0000          277.0       93.0000      637.0     27.00   
6       230.0   90.0000          115.0       45.0000      179.0     91.00   
7       377.0   49.0000          188.0       75.0000      385.0     98.00   
8       286.0   43.0000          143.0       22.0000      248.0     68.00   
9         NaN    1.0000            NaN           NaN        NaN       NaN   
10        NaN       NaN            NaN           NaN     1601.0     94.00   
11        1.0   46.7633            1.0       23.3833        1.0     63.34   

In [57]:
OccsTotalsNY = AllOccsNY.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsNY['# of people'] = OccsTotalsNY['occupation'].map(AllOccsNY['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsNY.to_string())

     occupation     6p_total  # of people
0        Broker  1776.120000            1
1        Farmer   674.310000            2
2       Mariner   377.490000            1
3      Merchant  4997.975500           21
4     Merchants  2450.827633            3
5     Physician   933.330000            1
6  Schoolmaster   131.150000            1


In [58]:
OccsTotalsNY = RemoveStopwords(OccsTotalsNY) 
print(OccsTotalsNY.to_string())

     occupation     6p_total  # of people
0        broker  1776.120000            1
1        farmer   674.310000            2
2       mariner   377.490000            1
3      merchant  4997.975500           21
4     merchants  2450.827633            3
5     physician   933.330000            1
6  schoolmaster   131.150000            1


In [59]:
PrintFutureOccupations(OccsTotalsNY['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


broker --> broker
farmer --> farmer
mariner --> mariner
merchant --> merchant
merchants --> merchants
physician --> physician
schoolmaster --> schoolmaster


In [60]:
OccsTotalsNY = MergeOccupations(OccsTotalsNY)

After Cleaning
--------------


     occupation     6p_total  # of people % of total people
0        broker  1776.120000            1              None
1        farmer   674.310000            2              None
2       mariner   377.490000            1              None
3      merchant  4997.975500           21              None
4     merchants  2450.827633            3              None
5        doctor   933.330000            1              None
6  schoolmaster   131.150000            1              None


After Removing Duplicates and Summing
-------------------------------------


     occupation     6p_total  # of people  % of total people
0        broker  1776.120000            1                  0
1        doctor   933.330000            1                  0
2        farmer   674.310000            2                  0
3       mariner   377.490000            1                  0
4      merchant  4997.975500           21                  0
5     merchants  2450.827633            3 

In [61]:
TurnToExcel(OccsTotalsNY, State)

# Pennsylvania

In [62]:
State = 'PA'
AllOccsPA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsPA = AllOccsPA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsPA)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0    102.14            1.0         51.07        1.0    117.34   
1       3212.0     38.00         1606.0         19.00     1400.0     66.00   
2       4529.0     96.00         2264.0         98.00     3312.0     47.00   
3        607.0      8.00          303.0         54.00      146.0     51.00   
4       2050.0     74.00         1025.0         37.00      745.0     57.00   
..         ...       ...            ...           ...        ...       ...   
409     9544.0     97.00         4772.0         48.00     8590.0     46.00   
410      880.0     75.00          440.0         38.00      753.0     91.00   
411       20.0     25.00           10.0         13.00        NaN       NaN   
412       43.0     60.00           21.0         80.00        NaN       NaN   
413      150.0       NaN           75.0           NaN      138.0     90.00   

                         town state occupation               co

In [63]:
OccsTotalsPA = AllOccsPA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsPA['# of people'] = OccsTotalsPA['occupation'].map(AllOccsPA['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsPA.to_string())

                                                         occupation     6p_total  # of people
0                                                        AH.Vat Law      58.0500            1
1        Adm of Alexander McCaraher late of Chester County Deceased     799.5100            1
2                            Administer of the estate of Job Vernon    1865.4400            1
3                                  Administor to the Est of J James      27.5300            1
4                                                   Attorney at Law    1223.6200            1
5                                                       Black Smith     692.2500            1
6                                                       Book Binder     783.0700            1
7                                                            Broker   64008.8400           60
8                                                             Clerk    1855.5800            4
9                                                           

In [64]:
OccsTotalsPA = RemoveStopwords(OccsTotalsPA) 
print(OccsTotalsPA.to_string())

                                               occupation     6p_total  # of people
0                                              ah.vat law      58.0500            1
1    adm alexander mccaraher late chester county deceased     799.5100            1
2                            administer estate job vernon    1865.4400            1
3                                  administor est j james      27.5300            1
4                                            attorney law    1223.6200            1
5                                             black smith     692.2500            1
6                                             book binder     783.0700            1
7                                                  broker   64008.8400           60
8                                                   clerk    1855.5800            4
9                                                      co    2524.3000            3
10                                            conveyancer     108.4000      

In [65]:
FindRemoveConst(OccsTotalsPA['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


ah.vat law --> ah.vat law
adm alexander mccaraher late chester county deceased --> adm alexander mccaraher late chester
administer estate job vernon --> administer estate
administor est j james --> administor est
attorney law --> attorney law
black smith --> black smith
book binder --> book binder
broker --> broker
clerk --> clerk
co --> co
conveyancer --> conveyancer
cordwainder --> cordwainder
druggist --> druggist
esquier --> esquier
esquire --> esquire
estate john benezet deceased --> estate john
estate --> estate
ex philadelphia county --> ex
ex william honeyman --> ex
ex estate john baird bucks co --> ex estate john baird
executers david grierlate y town --> executers david grierlate
executor estate abraham hull deceased --> executor estate abraham
executor estate mathias londenberger --> executor estate
executor estate doctor benjamin franklin deceased --> executor estate doctor benjamin
farmer --> f

In [66]:
PrintFutureOccupations(OccsTotalsPA['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


ah.vat law --> ah.vat
adm alexander mccaraher late chester --> adm
administer estate --> administer
administor est --> administor
attorney law --> attorney
black smith --> black
book binder --> book
broker --> broker
clerk --> clerk
co --> co
conveyancer --> conveyancer
cordwainder --> cordwainder
druggist --> druggist
esquier --> esquier
esquire --> esquire
estate john --> estate
estate --> estate
ex --> ex
ex --> ex
ex estate john baird --> baird
executers david grierlate --> executers
executor estate abraham --> executor
executor estate --> executor
executor estate doctor benjamin --> executor
farmer --> farmer
furrier --> furrier
gentleman --> gentleman
goldsmith --> goldsmith
guardian --> guardian
guardian --> guardian
guardian --> guardian
hatter --> hatter
inn keeper --> inn
junior broker --> junior
law --> law
lawyer --> lawyer
mariner --> mariner
merchant --> merchant
merchant execute

In [67]:
OccsTotalsPA = MergeOccupations(OccsTotalsPA)

After Cleaning
--------------


     occupation     6p_total  # of people % of total people
0      attorney      58.0500            1              None
1    administer     799.5100            1              None
2    administer    1865.4400            1              None
3    administor      27.5300            1              None
4      attorney    1223.6200            1              None
5    blacksmith     692.2500            1              None
6   book binder     783.0700            1              None
7        broker   64008.8400           60              None
8         clerk    1855.5800            4              None
9          None    2524.3000            3              None
10  conveyancer     108.4000            1              None
11  cordwainder     281.2500            1              None
12     druggist    4922.4300            2              None
13          esq   20672.0700            6              None
14          esq   20770.1700           15              None
15      

In [68]:
TurnToExcel(OccsTotalsPA, State)

# Rhode Island

In [69]:
State = 'RI'
AllOccsRI = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsRI = AllOccsRI.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsRI)

     6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0        154.0      20.0           77.0          10.0      192.0       NaN   
1        194.0      54.0           97.0          27.0      222.0       1.0   
2        359.0      50.0          179.0          76.0      254.0      21.0   
3          7.0      16.0            3.0          58.0       55.0      82.0   
4         35.0       2.0           17.0          51.0       38.0      94.0   
..         ...       ...            ...           ...        ...       ...   
443      164.0      96.0          165.0          27.0      100.0      22.0   
444       59.0      90.0           29.0          95.0       70.0       8.0   
445       28.0      44.0           14.0          22.0       29.0      70.0   
446      142.0      44.0           71.0          22.0      132.0      43.0   
447       33.0      33.0           17.0          39.0       31.0      94.0   

             town state occupation             county name_type

In [70]:
OccsTotalsRI = AllOccsRI.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsRI['# of people'] = OccsTotalsRI['occupation'].map(AllOccsRI['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsRI.to_string())

                                                                          occupation   6p_total  # of people
0                      Administratrix to the Estate of Joshua Brown Esquire Deceased       0.00            1
1                                                                        Auctioneers     460.77            3
2                                                                              Baker     206.37            1
3                                                                         Blacksmith     540.68            3
4                                                                         Blockmaker       4.43            1
5                                                                             Broker     171.50            2
6                                                                            Butcher     235.70            1
7                                                                          Carpenter       0.00            1
8                  

In [71]:
OccsTotalsRI = RemoveStopwords(OccsTotalsRI) 
print(OccsTotalsRI.to_string())

                                                       occupation   6p_total  # of people
0             administratrix estate joshua brown esquire deceased       0.00            1
1                                                     auctioneers     460.77            3
2                                                           baker     206.37            1
3                                                      blacksmith     540.68            3
4                                                      blockmaker       4.43            1
5                                                          broker     171.50            2
6                                                         butcher     235.70            1
7                                                       carpenter       0.00            1
8                                                    chaise maker      47.57            1
9                                                          cooper    1596.15            3
10        

In [72]:
FindRemoveConst(OccsTotalsRI['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


administratrix estate joshua brown esquire deceased --> administratrix estate joshua brown
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
broker --> broker
butcher --> butcher
carpenter --> carpenter
chaise maker --> chaise maker
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
esquier --> esquier
esquire --> esquire
extors late jn jenckes esq deceased --> extors late jn jenckes
extors late nicholas brown esq deceased --> extors late nicholas brown
farmer --> farmer
general treasurer state rhode island suceessors said office --> general treasurer state rhode island suceessors
gentleman --> gentleman
glazier --> glazier
goldsmith --> goldsmith
hatter --> hatter
housewright --> housewright
innholder --> innholder
labourer --> labourer
mariner --> mariner
mason --> mason
merchant --> merchant
merchants --> m

In [73]:
Remove = 1

In [74]:
PrintFutureOccupations(OccsTotalsRI['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administratrix estate joshua brown esquire --> administratrix
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
broker --> broker
butcher --> butcher
carpenter --> carpenter
chaise maker --> chaise
cooper --> cooper
cordwainer --> cordwainer
deacon --> deacon
distiller --> distiller
doctor --> doctor
esquier --> esquier
esquire --> esquire
extors late jn jenckes esq --> extors
extors late nicholas brown esq --> extors
farmer --> farmer
general treasurer state rhode island suceessors said --> general
gentleman --> gentleman
glazier --> glazier
goldsmith --> goldsmith
hatter --> hatter
housewright --> housewright
innholder --> innholder
labourer --> labourer
mariner --> mariner
mason --> mason
merchant --> merchant
merchants --> merchants
physician --> physician
reverend --> reverend
shipwright --> shipwright
shoemaker --> shoemaker
spinster --> spin

In [75]:
OccsTotalsRI = MergeOccupations(OccsTotalsRI)

After Cleaning
--------------


        occupation   6p_total  # of people % of total people
0   administratrix       0.00            1              None
1      auctioneers     460.77            3              None
2            baker     206.37            1              None
3       blacksmith     540.68            3              None
4       blockmaker       4.43            1              None
5           broker     171.50            2              None
6          butcher     235.70            1              None
7        carpenter       0.00            1              None
8     chaise maker      47.57            1              None
9           cooper    1596.15            3              None
10       shoemaker     417.48            1              None
11          deacon      16.35            1              None
12       distiller     466.25            3              None
13          doctor     445.45            1              None
14             esq      13.08            1           

In [76]:
TurnToExcel(OccsTotalsRI, State)

# South Carolina

In [77]:
State = 'SC'
AllOccsSC = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsSC = AllOccsSC.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsSC)

     6p_Dollar   6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0          1.0   824.4133            1.0      412.2033        1.0    941.54   
1          1.0  3987.9033            1.0     1993.9533        1.0   3625.67   
2         17.0   150.0000           13.0       75.0000       14.0    129.10   
3         17.0  1067.9200           13.0      533.9600       14.0   1165.92   
4         17.0   195.0200           13.0       97.5100       14.0    212.82   
..         ...        ...            ...           ...        ...       ...   
286        NaN        NaN            NaN           NaN      651.0     53.00   
287        NaN        NaN            NaN           NaN      140.0     77.00   
288        NaN        NaN            NaN           NaN      167.0     77.00   
289        NaN        NaN            NaN           NaN     3600.0       NaN   
290      180.0    47.0000           90.0       24.0000      207.0     93.00   

               town state       occupation         

In [78]:
OccsTotalsSC = AllOccsSC.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsSC['# of people'] = OccsTotalsSC['occupation'].map(AllOccsSC['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsSC.to_string())

                                      occupation      6p_total  # of people
0            Administrator to John Harleston Jun     33.330000            1
1                 Administrator to Joseph Warley   1671.440000            1
2                       Assignee of James Simons    345.020000            1
3                                Attorney at Law   3690.940000            9
4                                      Autioneer     10.650000            1
5                                   Blacks smith   1423.480000            1
6                                     Blockmaker   5264.230000            2
7                                     Bricklayer     37.350000            1
8                                         Broker   2334.890000           11
9                                  Cabinet Maker     74.460000            1
10                                     Carpenter    449.020000            5
11                          Charleston Merchants     26.670000            2
12          

In [79]:
OccsTotalsSC = RemoveStopwords(OccsTotalsSC) 
print(OccsTotalsSC.to_string())

                           occupation      6p_total  # of people
0    administrator john harleston jun     33.330000            1
1         administrator joseph warley   1671.440000            1
2               assignee james simons    345.020000            1
3                        attorney law   3690.940000            9
4                           autioneer     10.650000            1
5                        blacks smith   1423.480000            1
6                          blockmaker   5264.230000            2
7                          bricklayer     37.350000            1
8                              broker   2334.890000           11
9                       cabinet maker     74.460000            1
10                          carpenter    449.020000            5
11               charleston merchants     26.670000            2
12                       clerk senate   1425.820000            2
13                 comtroller customs     53.740000            1
14                       

In [80]:
FindRemoveConst(OccsTotalsSC['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


administrator john harleston jun --> administrator john harleston
administrator joseph warley --> administrator joseph
assignee james simons --> assignee james
attorney law --> attorney law
autioneer --> autioneer
blacks smith --> blacks smith
blockmaker --> blockmaker
bricklayer --> bricklayer
broker --> broker
cabinet maker --> cabinet maker
carpenter --> carpenter
charleston merchants --> charleston merchants
clerk senate --> clerk senate
comtroller customs --> comtroller customs
esq --> esq
excuetor --> excuetor
executor philip hawkins --> executor philip
executor benjamin tucker --> executor benjamin
executor jacob smiser --> executor jacob
executor james peart --> executor james
executor samuel bonneau --> executor samuel
executor sarah lampriere --> executor sarah
executor william gibbs --> executor william
executor william parker --> executor william
executor richard mason --> executor richard
execu

In [81]:
Remove = 1

In [82]:
PrintFutureOccupations(OccsTotalsSC['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


administrator john harleston --> administrator
administrator joseph --> administrator
assignee james --> assignee
attorney law --> attorney
autioneer --> autioneer
blacks smith --> blacks
blockmaker --> blockmaker
bricklayer --> bricklayer
broker --> broker
cabinet maker --> cabinet
carpenter --> carpenter
charleston merchants --> charleston
clerk senate --> clerk
comtroller customs --> comtroller
esq --> esq
excuetor --> excuetor
executor philip --> executor
executor benjamin --> executor
executor jacob --> executor
executor james --> executor
executor samuel --> executor
executor sarah --> executor
executor william --> executor
executor william --> executor
executor richard --> executor
executrix william --> executrix
factor --> factor
gentleman --> gentleman
hair dresser --> hair
mariner --> mariner
marriner --> marriner
merchant --> merchant
merchant executor benjamin --> merchant
merchant

In [83]:
OccsTotalsSC = MergeOccupations(OccsTotalsSC)

After Cleaning
--------------


        occupation      6p_total  # of people % of total people
0    administrator     33.330000            1              None
1    administrator   1671.440000            1              None
2         assignee    345.020000            1              None
3         attorney   3690.940000            9              None
4        autioneer     10.650000            1              None
5       blacksmith   1423.480000            1              None
6       blockmaker   5264.230000            2              None
7       bricklayer     37.350000            1              None
8           broker   2334.890000           11              None
9    cabinet maker     74.460000            1              None
10       carpenter    449.020000            5              None
11       merchants     26.670000            2              None
12           clerk   1425.820000            2              None
13      comtroller     53.740000            1              None
14      

In [84]:
TurnToExcel(OccsTotalsSC, State)

# Virginia

In [85]:
State = 'VA'
AllOccsVA = AllOccs[AllOccs['state'] == State] #Separate rows by state
AllOccsVA = AllOccsVA.dropna(subset=['occupation']).reset_index(drop=True) #Drop any rows w/o an occupation listed. 
print(AllOccsVA)

   6p_Dollar  6p_Cents  6p_def_Dollar  6p_def_Cents  3p_Dollar  3p_Cents  \
0       17.0  351.6233           13.0      175.8133       14.0    384.09   
1       78.0   33.0000           39.0       17.0000       42.0     30.00   
2       66.0   66.0000           33.0       33.0000       63.0     24.00   
3        3.0   82.0000            1.0       92.0000        3.0     74.00   
4       46.0    2.0000           23.0        1.0000       44.0     89.00   

                        town state occupation             county name_type  \
0        Portsmouth Virginia    VA  Merchants  Rockingham County      town   
1   Accomack County Virginia    VA  Gentleman    Accomack County    county   
2  Virginia and Philadelphia    VA   Merchant                NaN     state   
3  Virginia and Philadelphia    VA   Merchant                NaN     state   
4              Williamsburgh    VA     Weaver  Greenbrier County      town   

    6p_total  
0  20.516233  
1  78.330000  
2  66.660000  
3   3.820000  

In [86]:
OccsTotalsVA = AllOccsVA.groupby('occupation')['6p_total'].apply(sum).reset_index() #Combine duplicate occupations.
OccsTotalsVA['# of people'] = OccsTotalsVA['occupation'].map(AllOccsVA['occupation'].value_counts(dropna=False)) #Map # of people that belong to each occupation.
print(OccsTotalsVA.to_string())

  occupation   6p_total  # of people
0  Gentleman  78.330000            1
1   Merchant  70.480000            2
2  Merchants  20.516233            1
3     Weaver  46.020000            1


In [87]:
OccsTotalsVA = RemoveStopwords(OccsTotalsVA) 
print(OccsTotalsVA.to_string())

  occupation   6p_total  # of people
0  gentleman  78.330000            1
1   merchant  70.480000            2
2  merchants  20.516233            1
3     weaver  46.020000            1


In [88]:
FindRemoveConst(OccsTotalsVA['occupation'].tolist())

Occupation Before --> Occupation After
--------------------------------------


gentleman --> gentleman
merchant --> merchant
merchants --> merchants
weaver --> weaver


In [89]:
PrintFutureOccupations(OccsTotalsVA['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


gentleman --> gentleman
merchant --> merchant
merchants --> merchants
weaver --> weaver


In [90]:
OccsTotalsVA = MergeOccupations(OccsTotalsVA)

After Cleaning
--------------


  occupation   6p_total  # of people % of total people
0  gentleman  78.330000            1              None
1   merchant  70.480000            2              None
2  merchants  20.516233            1              None
3     weaver  46.020000            1              None


After Removing Duplicates and Summing
-------------------------------------


  occupation   6p_total  # of people  % of total people
0  gentleman  78.330000            1                  0
1   merchant  70.480000            2                  0
2  merchants  20.516233            1                  0
3     weaver  46.020000            1                  0


Removed: merchants --> merchant


After Using Fuzzy String Matching
---------------------------------


  occupation   6p_total  # of people  % of total people  average debt per person
0   merchant  90.996233            3               60.0                30.332078
1  gentleman  78.330000            1               20.0          

In [91]:
TurnToExcel(OccsTotalsVA, State)

# All States

In [92]:
#Append each states occupation + totals dataframes to the end of each other. 
OccsTotalsAll = pd.concat([OccsTotalsCT, OccsTotalsGA, OccsTotalsMD, OccsTotalsNC, OccsTotalsNH, OccsTotalsNJ,
                          OccsTotalsNY, OccsTotalsPA, OccsTotalsRI, OccsTotalsSC, OccsTotalsVA]).reset_index(drop=True)
print(OccsTotalsAll.to_string())

            occupation       6p_total  # of people  % of total people  average debt per person
0             merchant   63765.000000           63          13.043478              1012.142857
1               farmer   36780.360000          170          35.196687               216.355059
2            gentleman   16624.320000           32           6.625259               519.510000
3            treasurer   15387.150000           14           2.898551              1099.082143
4                  esq   13020.090000           13           2.691511              1001.545385
5             attorney   11114.600000           20           4.140787               555.730000
6               trader    9454.130000           46           9.523810               205.524565
7                widow    6488.110000           28           5.797101               231.718214
8             executor    4558.250000            7           1.449275               651.178571
9        adminastrator    4081.060000           10

In [93]:
OccsTotalsAll = OccsTotalsAll.groupby('occupation')[['6p_total', '# of people']].apply(sum).reset_index() #Combine duplicate occupations.
print(OccsTotalsAll.to_string())

           occupation       6p_total  # of people
0       adminastrator    4081.060000         10.0
1          administer    2692.480000          3.0
2       administrator    1727.270000          4.0
3      administratrix       0.000000          1.0
4          apothecary       7.720000          1.0
5            assignee     361.690000          2.0
6            attorney   24827.900000         35.0
7          auctioneer     123.190000          2.0
8         auctioneers     460.770000          3.0
9               baker    2365.410000          2.0
10         blacksmith    3436.000000          9.0
11         blockmaker    5268.660000          3.0
12        book binder     783.070000          1.0
13         bricklayer      37.350000          1.0
14             broker   79191.020000         80.0
15            butcher     235.700000          1.0
16      cabinet maker      74.460000          1.0
17          carpenter     784.300000         10.0
18            cashier      85.300000          2.0


In [94]:
PrintFutureOccupations(OccsTotalsAll['occupation'].tolist())

Occupations --> Potential Cleaned Occupations
---------------------------------------------


adminastrator --> adminastrator
administer --> administer
administrator --> administrator
administratrix --> administratrix
apothecary --> apothecary
assignee --> assignee
attorney --> attorney
auctioneer --> auctioneer
auctioneers --> auctioneers
baker --> baker
blacksmith --> blacksmith
blockmaker --> blockmaker
book binder --> book
bricklayer --> bricklayer
broker --> broker
butcher --> butcher
cabinet maker --> cabinet
carpenter --> carpenter
cashier --> cashier
chaise maker --> chaise
clerk --> clerk
clothier --> clothier
committee --> committee
comtroller --> comtroller
conveyancer --> conveyancer
cooper --> cooper
cordwainder --> cordwainder
deacon --> deacon
distiller --> distiller
doctor --> doctor
doctors --> doctors
druggist --> druggist
esq --> esq
estate --> estate
excuetor --> excuetor
executers --> executers
executor --> executor
executors --> executors
extors --> extors
farmer 

In [95]:
#Exclude these wors from any fuzzy string matching. Causes errors. 
NoMerge.append('planter')
NoMerge.append('printer')

In [96]:
OccsTotalsAll = MergeOccupations(OccsTotalsAll)

After Cleaning
--------------


           occupation       6p_total  # of people % of total people
0       adminastrator    4081.060000         10.0              None
1       adminastrator    2692.480000          3.0              None
2       administrator    1727.270000          4.0              None
3      administratrix       0.000000          1.0              None
4          apothecary       7.720000          1.0              None
5            assignee     361.690000          2.0              None
6            attorney   24827.900000         35.0              None
7          auctioneer     123.190000          2.0              None
8         auctioneers     460.770000          3.0              None
9               baker    2365.410000          2.0              None
10         blacksmith    3436.000000          9.0              None
11         blockmaker    5268.660000          3.0              None
12        book binder     783.070000          1.0              None
13         brick

In [97]:
#Remove any occupatins will a total of '0'
OccsTotalsAll = OccsTotalsAll[OccsTotalsAll['6p_total'] != 0].reset_index(drop=True)
OccsTotalsAll['% of total people'] = (OccsTotalsAll['# of people'] / OccsTotalsAll['# of people'].sum()) * 100 #Recalculate the '% of total people'.
print(OccsTotalsAll.to_string())

           occupation       6p_total  # of people  % of total people  average debt per person
0            merchant  975180.806095          669          34.898279              1457.669366
1           treasurer  251444.820000           32           1.669275              7857.650625
2                 esq  184573.597932          235          12.258738               785.419566
3              broker   79191.020000           80           4.173187               989.887750
4              farmer   59143.600000          274          14.293166               215.852555
5           gentleman   41367.310000           60           3.129890               689.455167
6               widow   31947.850000           80           4.173187               399.348125
7              doctor   25249.720000           44           2.295253               573.857273
8            attorney   24827.900000           35           1.825769               709.368571
9            excuetor   18219.350200           33           

In [98]:
TurnToExcel(OccsTotalsAll, 'AllStates')